In [42]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time

In [30]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")  # Run without GUI
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--log-level=3")

# Path to your WebDriver executable
webdriver_path = r'C:\Users\llq061\Anaconda3\envs\iSearch\datacollection\chromedriver-win64\chromedriver-win64\chromedriver.exe'
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service)

# Iteration 1

In [52]:
import re
import html
import time
from unidecode import unidecode 
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")  # Run without GUI
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--log-level=3")

# Path to your WebDriver executable
webdriver_path = r'C:\Users\llq061\Documents\chromedriver-win64\chromedriver-win64\chromedriver.exe'
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service)

url = 'https://www.cpso.on.ca/Public/Doctor-Search'
driver.get(url)

doctor_data = []

provine_pattern = r'\b(ON|BC|AB|NS|NL|NT|MB|YT|PE|QC|SK|NU)\b'

try:
    button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="p_lt_ctl01_pageplaceholder_p_lt_ctl02_CPSO_AllDoctorsSearch_btnSubmit"]'))
    )
    
    button.click()
    time.sleep(3)  
    
    current_page = 1
    last_page = 150
    
    while current_page <= last_page:
        results = driver.find_elements(By.CLASS_NAME, 'doctor-search-results--result')
        
        for result in results:
            doctor_info = {}
            
            def clean_html_content(html_content):
                text_only = re.sub(r'<[^>]*>', '', html_content)
                decoded_text = html.unescape(text_only)
                normalized_text = unicodedata.normalize('NFD', decoded_text)
                cleaned_text = ''.join([c for c in normalized_text if not unicodedata.combining(c)])
                return cleaned_text.replace('\xa0', ' ').strip()

            
            #Getting Doctors Name#
            h3_tag = result.find_element(By.TAG_NAME, 'h3').text
            doctor_name = h3_tag.split(' (')[0].strip()
            doctor_name = unicodedata.normalize('NFD', doctor_name)
            doctor_info['Doctors Name'] = doctor_name
            print(doctor_name)
            
            doctor_info['Last Name'] = doctor_name.split(',')[0].strip()
            #print(doctor_info['Last Name'])
            
            doctor_info['First Name'] = doctor_name.split(',')[-1].strip()
            #print(doctor_info['First Name'])
            
            doctor_info['CPSO Number'] = h3_tag.split('CPSO#: ')[-1].replace(')', '').strip()
            
            #There are alert for certain doctors so this code is extracting the alerts, wherever they are present#
            doc_alert = result.find_elements(By.CLASS_NAME, 'doc-alert')
            if doc_alert:
                #print("Concerns or Additional Info: ")
                    
                ul_tag = result.find_elements(By.TAG_NAME, 'ul')
                if ul_tag:
                    list_items = ul_tag[0].find_elements(By.TAG_NAME, 'li')
            
                    if list_items:
                        for item in list_items:
                            doctor_info['Concerns'] = item.text  
                    else:
                        print("No list items found in ul tag.")
                else:
                    print("No ul tag found.")
            else:
                print("No concerns or additional info.")
                
            
            practice_location = ""
            province = ""
            postal_code = ""
            specialization = ""
            cleaned_phone_number = ""
            fax_number = ""

            # Finding all the child elements in the article and print them in the correct order#
            child_elements = result.find_elements(By.XPATH, './*')
            
            for i, element in enumerate(child_elements):
                tag_name = element.tag_name
                
                #This is checking the h4 tag to extract the practice of location#
                if tag_name == 'h4':
                    if 'Location of Practice' in element.text:
                        next_p = child_elements[i + 1] if (i + 1) < len(child_elements) and child_elements[i + 1].tag_name == 'p' else None
                        if next_p:
                            
                            location_html = next_p.get_attribute('innerHTML')
                            location_lines = location_html.split('<br>')
                            
                            cleaned_location_lines = [clean_html_content(line) for line in location_lines if line.strip()]
                            practice_location = ' '.join([line for line in cleaned_location_lines if 'Phone:' not in line and 'Fax:' not in line])
                            
                            ##Getting clean Postal Code in a new column##
                            postal_code = practice_location[-7:].strip()
                            #print(practice_Pstlcode)
                            
                            ##Getting clean Province in a new column##
                            match = re.search(provine_pattern, practice_location)
                            if match:
                                province = match.group(0)
                                #print(province)
                                practice_location = practice_location.replace(province, '').strip()
                            
                            ##Practice location without Province and Postal Code##
                            practice_location = practice_location[:-10].strip()
                            #print(practice_location)
                            
                            
                            def clean_phone_number_with_extension(phone):
                                phone_pattern = re.compile(r'[^\d+]') 
                                phone_parts = re.split(r'(ext|x|extension)', phone, flags=re.IGNORECASE) 
                                cleaned_phone = phone_pattern.sub('', phone_parts[0]).strip()
                                if len(phone_parts) > 1:
                                    extension = ' '.join(phone_parts[1:]).strip()
                                    return f"{cleaned_phone} {extension}"
                                else:
                                    return cleaned_phone
                            
                            #clean/standardize phone number according to dtc table#
                            for line in cleaned_location_lines:
                                if 'Phone:' in line:
                                    raw_phone_number = line.replace('Phone:', '').strip()
                                    cleaned_phone_number = clean_phone_number_with_extension(raw_phone_number)
                                    #print(f'Cleaned Phone Number: {cleaned_phone_number}')
                                    doctor_info['Phone Number'] = cleaned_phone_number
                                    
                                elif 'Fax:' in line:
                                    raw_fax_number = line.replace('Fax:', '').strip()
                                    fax_number = clean_phone_number_with_extension(raw_fax_number)
                                    doctor_info['Fax Number'] = fax_number
                                    #print(fax_number)
                                
                    #This is checking the h4 tag to extract the area of specialization for doctors#        
                    elif 'Area(s) of Specialization' in element.text:
                        next_p = child_elements[i + 1] if (i + 1) < len(child_elements) and child_elements[i + 1].tag_name == 'p' else None
                        if next_p:
                            specialization = clean_html_content(next_p.get_attribute('innerHTML'))
            
            doctor_info["Location of Practice"] = practice_location.strip()
            doctor_info["Province"] = province.strip()
            doctor_info["Postal Code"] = postal_code.strip()
            doctor_info["Phone Number"] = cleaned_phone_number.strip()
            doctor_info["Fax Number"] = fax_number.strip()
            doctor_info["Specialization"] = specialization.strip()
            
            #This is appending all the data from data dictionary to the list created above#
            doctor_data.append(doctor_info)
             
        current_page += 1
        if current_page > last_page:
            break
        
        try:
            next_button_xpath = f'//button[@name="newPageNumber" and @value="{current_page}"]'
            
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, next_button_xpath))
            )
            
            next_button = driver.find_element(By.XPATH, next_button_xpath)
            next_button.click()
            time.sleep(2)
        except (NoSuchElementException, TimeoutException) as e:
            print(f"Failed to find link for page {current_page}: {e}")
            break
        
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    driver.quit()

df = pd.DataFrame(doctor_data)

df.to_csv('doctor_data.csv', index=False)

print("Data saved to doctor_data.csv")

Aalders, Ryan Francis
No concerns or additional info.
Aazami, Pejman
No concerns or additional info.
Abadir, Rony Michel Metry
No concerns or additional info.
Abawi, Blake
No concerns or additional info.
Abbarin, Alireza
No concerns or additional info.
Abbas, Ahmed
Abbasi, Alina
No concerns or additional info.
Abbasi, Iram Naz
No concerns or additional info.
Abbasiranjbar, Vida
No concerns or additional info.
Abbas, Myriam
No concerns or additional info.
Abbas, Wasan Abdul Ameer Abdul
No concerns or additional info.
Abbat, Bipandeep Kaur
Abba, Zaheera
No concerns or additional info.
Abbott, Laura Lea
No concerns or additional info.
Abboud, Rana
No concerns or additional info.
Abbud, Saad
No concerns or additional info.
Abd El Mesih, Sameh Abd El Mesih Nesim
No concerns or additional info.
Abdallah, Amel
No concerns or additional info.
Abdallah, Barwaaqo
No concerns or additional info.
Abdalla, Monika Adel Sobhy Morkos
Abdalla, Tarig Hamad Balla
No concerns or additional info.
Abdel Mal

Bignucolo, Stephen Joel
No concerns or additional info.
Bihi, Ali
Bihun, Julia Anne Patricia
No concerns or additional info.
Bilawski, Darlene Ann
No concerns or additional info.
Bilbily, Janan Fahmi Yacoub
No concerns or additional info.
Bilbily, Nicole
No concerns or additional info.
Bilcea, Carmen
No concerns or additional info.
Bilibajkich, Juliana
No concerns or additional info.
Bilibajkich, Mara Melinda
No concerns or additional info.
Bilic, Monika
Bilkstys-Richardson, Jokubas Gintaras
No concerns or additional info.
Billie, Christine
No concerns or additional info.
Billing, Suzanne Lindheimer
No concerns or additional info.
Bilodeau, Eric
Bimesl, Hamid
No concerns or additional info.
Biner, Mark Jacob
No concerns or additional info.
Binette, Andrew Thomas
No concerns or additional info.
Binette, Natalie Anne
No concerns or additional info.
Bing-Wo, Sandra Elaine
No concerns or additional info.
Biondi, Dax James
No concerns or additional info.
Bir, Amnit Singh
No concerns or addi

Bolla, Inderjit
No concerns or additional info.
Bolland, Mary Frances
No concerns or additional info.
Bollano, Enkelejda Teli
No concerns or additional info.
Bolla, Ramandeep Kaur
No concerns or additional info.
Bollinger, Megan Loren
No concerns or additional info.
Bolton, Craig James
No concerns or additional info.
Bolton, Gregg Maynard
No concerns or additional info.
Bolton, James Edward
No concerns or additional info.
Bolzon, Colin Michael
No concerns or additional info.
Bonacci, Giovanni
No concerns or additional info.
Bonacci, Giuseppe
No concerns or additional info.
Bonakdar, Hamid reza
Bond, Andrew James
No concerns or additional info.
Bondarev, Eugene
No concerns or additional info.
Bondar, Steven Michael
No concerns or additional info.
Bonder, Lauren Heather Levin
No concerns or additional info.
Bond, Geoffrey Hubert
No concerns or additional info.
Bond, Rebecca Mary
No concerns or additional info.
Bondy, Jennifer
No concerns or additional info.
Bongard, Alexandra Frances
No 

Aleksic, Mila
No concerns or additional info.
Aleksiejuk, Janusz
No concerns or additional info.
Alemayehu, Efrem
No concerns or additional info.
Brade, Sara Jessica
No concerns or additional info.
Bradford, Paul
No concerns or additional info.
Bradley, Jeffrey Robert
No concerns or additional info.
Bradley, John Terrence
No concerns or additional info.
Bradley, Lindsay Ann
No concerns or additional info.
Brady, Scott Ronald John
No concerns or additional info.
Braganza, David Andrew
No concerns or additional info.
Brahmbhatt, Ronak Kirankumar
No concerns or additional info.
Braidwood, Danielle
No concerns or additional info.
Braidwood, Mark Albert Thomas
No concerns or additional info.
Alexov, Lubomir
No concerns or additional info.
Aleykina, Natalya Oleksandrivna
No concerns or additional info.
Alfalah, Ibrahim Mohamed
No concerns or additional info.
Alfayad, Ammar
No concerns or additional info.
Alfayadh, Ridhab Ahmad Abdul Razzak
No concerns or additional info.
Alfiya, Ammar Jameel

Bruma, Cleopatra Virginia
No concerns or additional info.
Brunet, Anne-Marie
No concerns or additional info.
Brunet-Filion, Nicholas
No concerns or additional info.
Brunet, Melissa Pauline
No concerns or additional info.
Brunet, Nicole Marie Alvine
No concerns or additional info.
Brunet, Steven Ronald Scott
No concerns or additional info.
Brunetta, Jason Michael
No concerns or additional info.
Bruni, Adrianna Rosetta Louise
No concerns or additional info.
Bruni, Alison Michelle
No concerns or additional info.
Bruni, Christopher Arthur
No concerns or additional info.
Brunner, Reinhard
No concerns or additional info.
Bruno, Adrian Joseph
No concerns or additional info.
Bruno, Brigida Ashley
No concerns or additional info.
Brunskill, Megen Theresa
No concerns or additional info.
Brunton, Gillian Mary
No concerns or additional info.
Brunt, Susan Anne
No concerns or additional info.
Brush, Maurana Karin Tober
No concerns or additional info.
Bryan, Colin Francis
No concerns or additional inf

Ang, Vivienne
No concerns or additional info.
Anike, Livinus Ugochukwu Anayo
No concerns or additional info.
Ani, Nonso Chiedozie
Aniol, Michael Joseph
No concerns or additional info.
Aniol, Wojciech
No concerns or additional info.
Anis, Hanaa Nabil
No concerns or additional info.
Anjoree, Sheila
Ankita, N-A
No concerns or additional info.
Annisette, Spencer William
No concerns or additional info.
Annis, Robert William Frank
No concerns or additional info.
Anochie, Egorume Ufuoma
No concerns or additional info.
Anochie, Festus Ifeanyi
No concerns or additional info.
Anochili, Charles Chimaobi
No concerns or additional info.
Anokye, Chandra Adele
No concerns or additional info.
Anosike, Jide Emeka
No concerns or additional info.
An, Pamela Hyun-Joo
No concerns or additional info.
Ansari, Ali Hashsham
No concerns or additional info.
Ansari, Darakhshan Shahreen
No concerns or additional info.
Ansari, Farhat
No concerns or additional info.
Ansari, Hafsah Ahmad
Ansari, Imran
No concerns or 

Catania, Amy Margaret Kathleen
No concerns or additional info.
Catania, Philip John James
No concerns or additional info.
Cataudella, Julia Angela
No concerns or additional info.
Catford, Erika Adele
No concerns or additional info.
Cathers, Christopher Royden
No concerns or additional info.
Catton, Lianne Letitia
No concerns or additional info.
Cattran, Meeka Lee
No concerns or additional info.
Caturay, Michael John
No concerns or additional info.
Asham-Youssef, Amal Anwar
No concerns or additional info.
Ashbourne, Jessica Alexandra
No concerns or additional info.
Ashe, Gerald Patrick
No concerns or additional info.
Ashfaq, Bushra
No concerns or additional info.
Ashikian, Haig
No concerns or additional info.
Ashley, Jeffrey Miles
No concerns or additional info.
Ash, Marla Rochelle
No concerns or additional info.
Ashraf, Asif Eqbal
No concerns or additional info.
Ashraf, Faryal
No concerns or additional info.
Ashtarani, Peyvand
No concerns or additional info.
Ashton, Rosalind Julia Clar

Azadeh, Manizheh
No concerns or additional info.
AZAD, JASON Karn
No concerns or additional info.
Azad, Madhu Sanjay
No concerns or additional info.
Azam, Almas Lubna
No concerns or additional info.
Azer, Eman Abdel Messieh
No concerns or additional info.
Azer, Nivin Fahmy Tawfik
No concerns or additional info.
Azer, Nivine Nadi Zaki
No concerns or additional info.
Azer, Raed
No concerns or additional info.
Azer, Salwa
No concerns or additional info.
Azharuddin, Tahera
Azher, Zaira
No concerns or additional info.
Azi, Adaku Michelle
No concerns or additional info.
Azimi, Kousha
No concerns or additional info.
Azimzadeh, Shahnaz
No concerns or additional info.
Aziz, Aliza
No concerns or additional info.
Aziz, Alnoor
No concerns or additional info.
Aziz, Brittany Laurel
No concerns or additional info.
Azizi Namini, Parastoo
No concerns or additional info.
Azizi, Hawmid
No concerns or additional info.
Aziz, Mohammed
No concerns or additional info.
Aziz, Natasha Ruth
No concerns or additio

No concerns or additional info.
Cheng, Man-Fai Frederick
No concerns or additional info.
Cheng, Michael Tak-Woo
No concerns or additional info.
Cheng, Michelle
No concerns or additional info.
Cheng, Richard Shing Sou
No concerns or additional info.
Cheng, Vivian Kwok-Ling
No concerns or additional info.
Cheng, Willoon Wai Ming
No concerns or additional info.
Chen, Hillary Joy
No concerns or additional info.
Chen, Howard Chun-Hao
No concerns or additional info.
Chen, Ing Je
No concerns or additional info.
Chen, Irene Wen-Ting
No concerns or additional info.
Chen, Jacky
No concerns or additional info.
Chen, Jacky Seng Leong
No concerns or additional info.
Chen, Jeyla Xiao Meng
No concerns or additional info.
Chen, Jingshu
No concerns or additional info.
Chen, Jingwen
No concerns or additional info.
Chen, John Shang-Che
No concerns or additional info.
Chen, Karen Guangxin
No concerns or additional info.
Chen, Kelly Kai-Li
No concerns or additional info.
Chen, Kuan-Chin Jean
No concerns or

Choi, Cyu-Chul
No concerns or additional info.
Choi, Deung Young
No concerns or additional info.
Choi, Ellen Hon-May
No concerns or additional info.
Choi-Fung, Betty Lai See
No concerns or additional info.
Choi, Hong-Yung David
No concerns or additional info.
Choi, Jacquelyn Shuk Wah
No concerns or additional info.
Choi, James Han
No concerns or additional info.
Choi, Ji Hyeon
No concerns or additional info.
Choi, Nicole Haejin
No concerns or additional info.
Choi, Steve Chi Sheung
No concerns or additional info.
Choi, Yoo Jung
No concerns or additional info.
Cho, Jae-Hoon Hoon
No concerns or additional info.
Cho, Kevin Yeon
No concerns or additional info.
Chomienne, Marie-Helene
No concerns or additional info.
Chong, Edison
No concerns or additional info.
Chong, Esther
No concerns or additional info.
Chong, Evan Richard
No concerns or additional info.
Chong, Joseph Ming
No concerns or additional info.
Chong, Laura Jill
No concerns or additional info.
Chong, Nancy Leta
No concerns or a